In [1]:
import pandas as pd
import numpy as np
import googlemaps
from datetime import datetime
import acquire_r
from env import google_key, google_key1
from googlemaps import convert
import wrangle_zillow_r
import wrangle_zillow
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import gmaps
import gmaps.datasets
gmaps.configure(api_key=google_key1) # Fill in with your API key
%matplotlib inline
import zillow_fe
import feature_selection
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import median_absolute_error, r2_score, mean_absolute_error
from math import sqrt
from sklearn.preprocessing import PolynomialFeatures
from pygam import LogisticGAM, LinearGAM, s, f, te
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats
import explore
import model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Read CSV file which was saved after data wrangling
zillow = pd.read_csv('zillow_fe.csv')
orig_rows = zillow.shape[0]
orig_cols = zillow.shape[1]
print (f'There are {orig_rows} rows and {orig_cols} columns in the zillow dataframe.')

There are 64747 rows and 48 columns in the zillow dataframe.


In [3]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64747 entries, 0 to 64746
Data columns (total 48 columns):
Unnamed: 0                        64747 non-null int64
bathroomcnt                       64747 non-null int64
bedroomcnt                        64747 non-null int64
finishedsquarefeet12              64747 non-null int64
garagecarcnt                      64747 non-null int64
garagetotalsqft                   64747 non-null int64
hashottuborspa                    64747 non-null int64
latitude                          64747 non-null float64
longitude                         64747 non-null float64
lotsizesquarefeet                 64747 non-null int64
poolcnt                           64747 non-null int64
poolsizesum                       64747 non-null int64
numberofstories                   64747 non-null int64
structuretaxvaluedollarcnt        64747 non-null int64
taxvaluedollarcnt                 64747 non-null int64
taxamount                         64747 non-null int64
taxdeli

In [12]:
X = train[['finishedsquarefeet12']]
train, X_scaled, scaler, kmeans, centroids = explore.cluster(train, X, 4)
#explore.scatter_plot('logerror','finishedsquarefeet12',zillow,kmeans, X_scaled, scaler)

In [16]:
#fix median_income coversion from object to float
#zillow = wrangle_zillow_r.fix_med(zillow)
zillow['living_area'] = zillow.lotsizesquarefeet/zillow.finishedsquarefeet12

In [17]:
# split data in train/test (80/20)
train, test = wrangle_zillow.split_my_data(zillow, 0.8)

In [18]:
train.shape, test.shape

((51797, 49), (12950, 49))

In [19]:
# Split the train into train and validate sets (80/20)
train, validate = wrangle_zillow.split_my_data(train, 0.8)

In [20]:
#check train and validate shapes
train.shape, validate.shape

((41437, 49), (10360, 49))

In [21]:
#add onehot encoding if needed
#function in model.py
# add age as a variable

X = train[['finishedsquarefeet12']]
train, X_scaled, scaler, kmeans, centroids = explore.cluster(train, X, 4)
#explore.scatter_plot('logerror','finishedsquarefeet12',train,kmeans, X_scaled, scaler)

# Modeling

### Baseline


In [15]:
# Lets first establish a baseline

zillow["base_logerror"] = zillow.logerror.mean()

RMSE_base = sqrt(mean_squared_error(zillow.logerror,zillow.base_logerror))

print(f'The Baseline RMSE logerror is :{RMSE_base}')

The Baseline RMSE logerror is :0.16322244734951857


In [16]:
# Lets first establish a baseline

test["base_logerror"] = test.logerror.mean()
RMSE_test = sqrt(mean_squared_error(test.logerror,test.base_logerror))
R2_test = r2_score(test.logerror,test.base_logerror)
# print(f'The Baseline RMSE logerror is :{RMSE_test}')
# print(f'The Baseline R2 logerror is :{R2_test}')

In [24]:
train.cluster.value_counts()

cluster_3    15274
cluster_1    13862
cluster_0     8595
cluster_2     3706
Name: cluster, dtype: int64

***

### Polynomial Regression Method


In [13]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
RMSE, R2 = model.poly_regression(train, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.161 and R2 is 0.032


In [14]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
RMSE, R2 = model.poly_regression(validate, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.161 and R2 is 0.112


***

### Simple Linear Regression Method

In [15]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
RMSE, R2 = model.linear_reg(train, features)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.163 and R2 is 0.002


In [16]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
RMSE, R2 = model.linear_reg(validate, features)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.171 and R2 is 0.002


***

### GAM Model

In [15]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']

model.GAM_model(train, features)

100% (11 of 11) |########################| Elapsed Time: 0:00:11 Time:  0:00:11


(0.161, 0.004)

In [16]:
model.GAM_model(validate, features)

100% (11 of 11) |########################| Elapsed Time: 0:00:02 Time:  0:00:02


(0.162, 0.006)

In [17]:
model.GAM_model(test, features)

100% (11 of 11) |########################| Elapsed Time: 0:00:03 Time:  0:00:03


(0.17, 0.007)

***
### Run all functions together with a single function

In [12]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']

summary_results_train = model.run_all_functions(train,features)
summary_results_train

100% (11 of 11) |########################| Elapsed Time: 0:00:13 Time:  0:00:13


,RMSE,R2
Baseline,0.163,0.000
Linear_model,0.163,0.002
polynomial_LR,0.161,0.032
GAM,0.163,0.004


In [13]:
summary_results_validate = model.run_all_functions(validate,features)
summary_results_validate

100% (11 of 11) |########################| Elapsed Time: 0:00:03 Time:  0:00:03


,RMSE,R2
Baseline,0.171,0.000
Linear_model,0.171,0.002
polynomial_LR,0.161,0.112
GAM,0.171,0.007


#### Seems like polynomial linear regression model performed best. Lets run the test dataset on polynomial LR model

In [20]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
RMSE, R2 = model.poly_regression(test, features,6)
print(f' The Test baseline RMSE is : {RMSE_test} and R2 is {R2_test}')
print(f' The model RMSE is : {RMSE} and R2 is {R2}')

 The Test baseline RMSE is : 0.15607026290260517 and R2 is 0.0
 The model RMSE is : 0.145 and R2 is 0.132


In [22]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
train_0 = train[train.cluster == 'cluster_0']
RMSE, R2, y_train0= model.poly_regression(train_0, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.148 and R2 is 0.095


In [23]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
train_1 = train[train.cluster == 'cluster_1']
RMSE, R2,y_train1 = model.poly_regression(train_1, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.155 and R2 is 0.081


In [24]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
train_2 = train[train.cluster == 'cluster_2']
RMSE, R2, y_train2 = model.poly_regression(train_2, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.158 and R2 is 0.326


In [25]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
train_3 = train[train.cluster == 'cluster_3']
RMSE, R2, y_train3 = model.poly_regression(train_3, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.157 and R2 is 0.058


In [26]:
 y_train_comb= pd.concat([y_train0, y_train1,y_train2,y_train3])

In [27]:
RMSE_train = sqrt(mean_squared_error(y_train_comb.logerror,y_train_comb.predicted_poly))
R2_train = r2_score(y_train_comb.logerror,y_train_comb.predicted_poly)

In [23]:
RMSE_train, R2_train

(0.1546663833400847, 0.10400346660235071)

In [27]:
# Fit kmeans on validate 
X = validate[['finishedsquarefeet12','logerror']]
scaler = MinMaxScaler(copy=True, feature_range=(0,1)).fit(X)
X_scaled = pd.DataFrame(scaler.transform(X), columns=X.columns.values).set_index([X.index.values])
kmeans.predict(X_scaled)
validate['cluster'] = kmeans.predict(X_scaled)
validate['cluster'] = 'cluster_' + validate.cluster.astype(str)

In [29]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
validate_0 = validate[validate.cluster == 'cluster_0']
RMSE, R2, y_validate0= model.poly_regression(validate_0, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.133 and R2 is 0.547


In [30]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
validate_1 = validate[validate.cluster == 'cluster_1']
RMSE, R2, y_validate1= model.poly_regression(validate_1, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.148 and R2 is 0.197


In [31]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
validate_2 = validate[validate.cluster == 'cluster_2']
RMSE, R2, y_validate2= model.poly_regression(validate_2, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.039 and R2 is 0.962


In [32]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
validate_3 = validate[validate.cluster == 'cluster_3']
RMSE, R2, y_validate3= model.poly_regression(validate_3, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.137 and R2 is 0.218


In [33]:
 y_validate_comb= pd.concat([y_validate0, y_validate1,y_validate2,y_validate3])

In [34]:
RMSE_validate = sqrt(mean_squared_error(y_validate_comb.logerror,y_validate_comb.predicted_poly))
R2_validate = r2_score(y_validate_comb.logerror,y_validate_comb.predicted_poly)

In [35]:
RMSE_validate, R2_validate 

(0.13668867253334133, 0.3615934886828699)

In [36]:
# Fit kmeans on test
X = test[['finishedsquarefeet12','logerror']]
scaler = MinMaxScaler(copy=True, feature_range=(0,1)).fit(X)
X_scaled = pd.DataFrame(scaler.transform(X), columns=X.columns.values).set_index([X.index.values])
kmeans.predict(X_scaled)
test['cluster'] = kmeans.predict(X_scaled)
test['cluster'] = 'cluster_' + test.cluster.astype(str)

In [38]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
test_0 = test[test.cluster == 'cluster_0']
RMSE, R2, y_test0= model.poly_regression(test_0, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.109 and R2 is 0.382


In [ ]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
test_1 = test[test.cluster == 'cluster_1']
RMSE, R2, y_test1= model.poly_regression(test_1, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

In [39]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
test_1 = test[test.cluster == 'cluster_1']
RMSE, R2, y_test1= model.poly_regression(test_1, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.147 and R2 is 0.223


In [40]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
test_2 = test[test.cluster == 'cluster_2']
RMSE, R2, y_test2= model.poly_regression(test_2, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.073 and R2 is 0.87


In [41]:
features = ['bathroomcnt', 'bedroomcnt', 'finishedsquarefeet12', 'taxvaluedollarcnt', 'median_income', 'living_area']
test_3 = test[test.cluster == 'cluster_3']
RMSE, R2, y_test3 = model.poly_regression(test_3, features,6)
print(f' The RMSE is : {RMSE} and R2 is {R2}')

 The RMSE is : 0.118 and R2 is 0.272


In [42]:
 y_test_comb= pd.concat([y_test0, y_test1,y_test2,y_test3])

In [44]:
RMSE_test = sqrt(mean_squared_error(y_test_comb.logerror,y_test_comb.predicted_poly))
R2_validate = r2_score(y_test_comb.logerror,y_test_comb.predicted_poly)

In [46]:
RMSE_test, R2_validate

(0.12585256247538126, 0.34974484872986444)